In [29]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# ===================== LOAD AND PREPROCESS DATA =====================
df = pd.read_excel(r"C:\nishit\UNi_MCTI\Threat intel\Ass1\Project\Submission4\Group11_output.xlsx")
df['Opcodes'] = df['Opcodes'].fillna("").astype(str)

In [31]:
# ===================== 1-GRAM PROCESSING =====================
unique_1grams = set()
for opcodes in df['Opcodes']:
    unique_1grams.update(op.strip() for op in opcodes.split(', '))
unique_1grams = sorted(unique_1grams)

one_gram_counts_df = pd.DataFrame(0, index=range(len(df)), columns=unique_1grams)
for i, row in df.iterrows():
    opcode_list = [op.strip() for op in row['Opcodes'].split(', ')]
    counts_1gram = Counter(opcode_list)
    for opcode, count in counts_1gram.items():
        one_gram_counts_df.at[i, opcode] = count

x_1gram = one_gram_counts_df.values
y_1gram = df['APT Group'].values


In [32]:
# ===================== 2-GRAM PROCESSING =====================
def generate_2grams(sequence):
    """Generate 2-grams from a sequence of opcodes."""
    return ['{'+', '.join(sequence[i:i + 2])+'}' for i in range(len(sequence) - 1)]

unique_2grams = set()
for opcodes in df['Opcodes']:
    opcode_list = [op.strip() for op in opcodes.split(', ')]
    two_grams = generate_2grams(opcode_list)
    unique_2grams.update(two_grams)
unique_2grams = sorted(unique_2grams)

two_gram_counts_df = pd.DataFrame(0, index=range(len(df)), columns=unique_2grams)
for i, row in df.iterrows():
    opcode_list = [op.strip() for op in row['Opcodes'].split(', ')]
    two_grams = generate_2grams(opcode_list)
    counts_2gram = Counter(two_grams)
    for two_gram, count in counts_2gram.items():
        two_gram_counts_df.at[i, two_gram] = count

x_2gram = two_gram_counts_df.values

In [33]:
# ===================== BALANCE DATA WITH SMOTE =====================
class_counts = pd.Series(y_1gram).value_counts()
classes_to_keep = class_counts[class_counts >= 3].index
df_filtered = df[df['APT Group'].isin(classes_to_keep)]

x_1gram_filtered = one_gram_counts_df.loc[df_filtered.index].values
y_1gram_filtered = df_filtered['APT Group'].values
x_2gram_filtered = two_gram_counts_df.loc[df_filtered.index].values
y_2gram_filtered = df_filtered['APT Group'].values

smote_k_neighbors = max(1, min(class_counts[class_counts >= 3].min() - 1, 5))

smote = SMOTE(k_neighbors=smote_k_neighbors, random_state=42)
x_1gram_balanced, y_1gram_balanced = smote.fit_resample(x_1gram_filtered, y_1gram_filtered)
x_2gram_balanced, y_2gram_balanced = smote.fit_resample(x_2gram_filtered, y_2gram_filtered)


In [34]:
# ===================== SPLIT AND SCALE DATA =====================
scaler = MinMaxScaler()
x_train_1gram, x_test_1gram, y_train_1gram, y_test_1gram = train_test_split(
    x_1gram_balanced, y_1gram_balanced, test_size=0.3, random_state=42
)
x_train_2gram, x_test_2gram, y_train_2gram, y_test_2gram = train_test_split(
    x_2gram_balanced, y_2gram_balanced, test_size=0.3, random_state=42
)

x_train_1gram_scaled = scaler.fit_transform(x_train_1gram)
x_test_1gram_scaled = scaler.transform(x_test_1gram)
x_train_2gram_scaled = scaler.fit_transform(x_train_2gram)
x_test_2gram_scaled = scaler.transform(x_test_2gram)

In [35]:
# ===================== MODEL IMPLEMENTATION =====================

# Define function to run models and print detailed performance metrics
def run_model(name, model, x_train, y_train, x_test, y_test, params):
    grid_search = GridSearchCV(model, params, cv=5)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test)

    # Performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred, normalize='true')

    print(f"\n{name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Normalized Confusion Matrix:")
    for row in cm:
        print(" ".join(f"{val:.2f}" for val in row))

    return best_model

# ===== MODEL IMPLEMENTATION =====

In [36]:
# Define function to run models and print detailed performance metrics
def run_model(name, model, x_train, y_train, x_test, y_test, params):
    grid_search = GridSearchCV(model, params, cv=5)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test)

    # Performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred, normalize='true')

    print(f"\n{name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Normalized Confusion Matrix:")
    for row in cm:
        print(" ".join(f"{val:.2f}" for val in row))

    return best_model

# ===== KNN, SVM, and Decision Tree SEPARATE =====

In [37]:
# KNN (1-gram)
knn_params = {'n_neighbors': [3], 'weights': ['uniform', 'distance']}
print("\n=== KNN with 1-gram ===")
best_knn_1gram = run_model(
    "KNN (1-gram)", KNeighborsClassifier(), x_train_1gram_scaled, y_train_1gram, x_test_1gram_scaled, y_test_1gram, knn_params
)


=== KNN with 1-gram ===

KNN (1-gram) Performance:
Accuracy: 0.8750
Recall: 0.8750
Precision: 0.8889
F1-Score: 0.8711
Normalized Confusion Matrix:
0.71 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.14 0.07 0.00
0.03 0.55 0.03 0.00 0.00 0.10 0.00 0.00 0.00 0.00 0.00 0.29
0.00 0.04 0.96 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.95 0.00 0.00 0.05 0.00
0.07 0.00 0.00 0.00 0.00 0.14 0.00 0.00 0.71 0.00 0.00 0.07
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00
0.00 0.00 0.06 0.00 0.00 0.06 0.06 0.00 0.06 0.00 0.00 0.76


In [38]:
# KNN (2-gram)
print("\n=== KNN with 2-gram ===")
best_knn_2gram = run_model(
    "KNN (2-gram)", KNeighborsClassifier(), x_train_2gram_scaled, y_train_2gram, x_test_2gram_scaled, y_test_2gram, knn_params
)



=== KNN with 2-gram ===

KNN (2-gram) Performance:
Accuracy: 0.8705
Recall: 0.8705
Precision: 0.8843
F1-Score: 0.8709
Normalized Confusion Matrix:
0.86 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.00 0.07
0.00 0.65 0.00 0.00 0.00 0.19 0.00 0.00 0.00 0.00 0.00 0.16
0.00 0.21 0.79 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.14 0.00 0.00 0.79 0.00 0.00 0.07
0.06 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.94 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.00 0.00 0.00 0.93 0.00
0.00 0.06 0.00 0.00 0.00 0.18 0.06 0.00 0.00 0.00 0.06 0.65


In [39]:
# SVM (1-gram)
svm_params = {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
print("\n=== SVM with 1-gram ===")
best_svm_1gram = run_model(
    "SVM (1-gram)", SVC(), x_train_1gram_scaled, y_train_1gram, x_test_1gram_scaled, y_test_1gram, svm_params
)


=== SVM with 1-gram ===

SVM (1-gram) Performance:
Accuracy: 0.9018
Recall: 0.9018
Precision: 0.9119
F1-Score: 0.8997
Normalized Confusion Matrix:
0.79 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.14 0.00 0.00
0.03 0.74 0.03 0.00 0.00 0.16 0.00 0.00 0.00 0.00 0.00 0.03
0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.95 0.05 0.00 0.00 0.00
0.07 0.00 0.07 0.00 0.00 0.14 0.07 0.00 0.57 0.07 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00
0.07 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.86 0.00
0.06 0.00 0.00 0.00 0.00 0.00 0.06 0.00 0.00 0.00 0.00 0.88


In [40]:
# SVM (2-gram)
print("\n=== SVM with 2-gram ===")
best_svm_2gram = run_model(
    "SVM (2-gram)", SVC(), x_train_2gram_scaled, y_train_2gram, x_test_2gram_scaled, y_test_2gram, svm_params
)


=== SVM with 2-gram ===

SVM (2-gram) Performance:
Accuracy: 0.9330
Recall: 0.9330
Precision: 0.9378
F1-Score: 0.9317
Normalized Confusion Matrix:
1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.81 0.10 0.00 0.00 0.03 0.00 0.00 0.00 0.00 0.03 0.03
0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00
0.00 0.00 0.07 0.00 0.00 0.14 0.00 0.00 0.71 0.00 0.00 0.07
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.00 0.07 0.00 0.86 0.00
0.00 0.00 0.06 0.00 0.00 0.00 0.06 0.00 0.00 0.00 0.06 0.82


In [41]:
# Decision Tree (1-gram)
dt_params = {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
print("\n=== Decision Tree with 1-gram ===")
best_dt_1gram = run_model(
    "Decision Tree (1-gram)", DecisionTreeClassifier(), x_train_1gram_scaled, y_train_1gram, x_test_1gram_scaled, y_test_1gram, dt_params
)


=== Decision Tree with 1-gram ===

Decision Tree (1-gram) Performance:
Accuracy: 0.8616
Recall: 0.8616
Precision: 0.8643
F1-Score: 0.8564
Normalized Confusion Matrix:
0.50 0.07 0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.14 0.21 0.00
0.03 0.68 0.03 0.00 0.00 0.06 0.00 0.00 0.06 0.00 0.03 0.10
0.00 0.04 0.96 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.89 0.05 0.00 0.05 0.00
0.07 0.00 0.00 0.00 0.00 0.07 0.00 0.00 0.86 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.00 0.00 0.07 0.86 0.00
0.06 0.06 0.06 0.00 0.00 0.06 0.06 0.00 0.06 0.00 0.06 0.59


In [42]:
# Decision Tree (2-gram)
print("\n=== Decision Tree with 2-gram ===")
best_dt_2gram = run_model(
    "Decision Tree (2-gram)", DecisionTreeClassifier(), x_train_2gram_scaled, y_train_2gram, x_test_2gram_scaled, y_test_2gram, dt_params
)



=== Decision Tree with 2-gram ===

Decision Tree (2-gram) Performance:
Accuracy: 0.8750
Recall: 0.8750
Precision: 0.8815
F1-Score: 0.8731
Normalized Confusion Matrix:
0.71 0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.07 0.07 0.07 0.00
0.00 0.65 0.06 0.00 0.00 0.00 0.00 0.03 0.10 0.00 0.00 0.16
0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.71 0.07 0.07 0.14
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00
0.00 0.14 0.00 0.00 0.00 0.00 0.00 0.00 0.07 0.00 0.71 0.07
0.00 0.06 0.12 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.12 0.71
